In [2]:
!python -m pip install setfit --q

In [3]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

In [4]:
# Load a dataset from the Hugging Face Hub
dataset = load_dataset("hebashakeel/dataset_wellness")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


README.md:   0%|          | 0.00/514 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.1k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/22.1k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/22.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/990 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/212 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/213 [00:00<?, ? examples/s]

In [39]:
num_classes = 6
train_dataset = dataset["train"].shuffle(seed=42).select(range(16 * num_classes))
test_dataset = dataset["test"]
eval_dataset = dataset['validation']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
# train_dataset['Text']

In [41]:
from huggingface_hub import notebook_login
notebook_login()

In [42]:
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    labels=["Intellectual Aspect", "Vocational Aspect", "Spiritual Aspect", "Physical Aspect", "Social Aspect", "Emotional Aspect"],
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [43]:
from setfit import Trainer, TrainingArguments

In [44]:
args = TrainingArguments(
    batch_size=8,
    num_epochs=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [45]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    metric="accuracy",
    column_mapping={"Text": "text", "Aspect": "label"}  # Map dataset columns to text/label expected by trainer
)


Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


In [46]:
eval_dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Dataset({
    features: ['Text', 'Aspect'],
    num_rows: 212
})

In [47]:
import wandb
wandb.init(mode='disabled')

/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py:202: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  settings = self._wl.settings.copy()


In [48]:
# Train and evaluate
trainer.train()

***** Running training *****
  Num unique pairs = 7474
  Batch size = 8
  Num epochs = 2


Epoch,Training Loss,Validation Loss
1,0.000200,0.220153
2,0.000100,0.222912


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [49]:
metrics = trainer.evaluate(eval_dataset)
print(metrics)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Applying column mapping to the evaluation dataset
***** Running evaluation *****


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

{'accuracy': 0.5990566037735849}


In [50]:
metrics = trainer.evaluate(test_dataset)
print(metrics)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Applying column mapping to the evaluation dataset
***** Running evaluation *****


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

{'accuracy': 0.6431924882629108}


In [51]:
# Push model to the Hub
trainer.push_to_hub("hebashakeel/wellness")

# Download from Hub
model = SetFitModel.from_pretrained("hebashakeel/wellness")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model_head.pkl:   0%|          | 0.00/37.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/37.8k [00:00<?, ?B/s]